In [13]:
from minio import Minio
from minio.error import S3Error

In [2]:
#client = Minio("lay.min.io:9443/", "admin", "e372e705-b435-4aee-8c49-1fa1401b3ec9", secure=True)

In [3]:
#client = Minio("play.min.io:9000", "Q3AM3UQ867SPQQA43P2F", "zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG", secure=True)

In [4]:
#obj_business = client.get_object("bronze", "EXP_1997.csv")

In [5]:
 #print(client.bucket_exists('bronze'))

In [14]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession


In [15]:
conf = ( 
    SparkConf()
            .setMaster("k8s://https://kubernetes.default.svc.cluster.local:443")  
            .setAppName("spark") 
            .set("spark.kubernetes.container.image", "magaiwer/pyspark-notebook:3.2.0") 
            .set("spark.kubernetes.namespace", "spark") 
            .set("spark.kubernetes.authenticate.caCertFile", "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt") 
            .set("spark.kubernetes.authenticate.oauthTokenFile", "/var/run/secrets/kubernetes.io/serviceaccount/token") 
            .set("spark.executor.instances", "3") 
            .set("spark.executor.cores", "1") 
            .set("spark.driver.memory", "1024m") 
            .set("spark.executor.memory", "1024m") 
            .set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark-sa") 
            .set("spark.kubernetes.authenticate.serviceAccountName", "spark-sa") 
            .set("spark.driver.port", "29413") 
            .set("spark.driver.host", "jupyter-labs.spark.svc.cluster.local") 
            .set("spark.driver.bindAddress", "0.0.0.0") 
            .set("spark.hadoop.com.amazonaws.services.s3.enableV4", "true") 
) 

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "Q3AM3UQ867SPQQA43P2F")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG")
#sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://10.32.0.5:9000") 
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "https://play.min.io:9000") 
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.fast.upload", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "2000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "2000")
sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled","false")



In [ ]:
NCM = spark.read.csv("s3a://bronze/NCM_SH.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")

In [34]:
sc.setLogLevel("OFF")
NCM.printSchema()

NCM.show()

root
 |-- CO_SH6: integer (nullable = true)
 |-- NO_SH6_POR: string (nullable = true)
 |-- NO_SH6_ESP: string (nullable = true)
 |-- NO_SH6_ING: string (nullable = true)
 |-- CO_SH4: string (nullable = true)
 |-- NO_SH4_POR: string (nullable = true)
 |-- NO_SH4_ESP: string (nullable = true)
 |-- NO_SH4_ING: string (nullable = true)
 |-- CO_SH2: string (nullable = true)
 |-- NO_SH2_POR: string (nullable = true)
 |-- NO_SH2_ESP: string (nullable = true)
 |-- NO_SH2_ING: string (nullable = true)
 |-- CO_NCM_SECROM: string (nullable = true)
 |-- NO_SEC_POR: string (nullable = true)
 |-- NO_SEC_ESP: string (nullable = true)
 |-- NO_SEC_ING: string (nullable = true)



+------+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+------+-------------+--------------+------------+-------------+--------------------+--------------------+--------------------+
|CO_SH6|          NO_SH6_POR|          NO_SH6_ESP|          NO_SH6_ING|CO_SH4|          NO_SH4_POR|          NO_SH4_ESP|          NO_SH4_ING|CO_SH2|   NO_SH2_POR|    NO_SH2_ESP|  NO_SH2_ING|CO_NCM_SECROM|          NO_SEC_POR|          NO_SEC_ESP|          NO_SEC_ING|
+------+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+------+-------------+--------------+------------+-------------+--------------------+--------------------+--------------------+
| 10110|Animais vivos das...|Caballos y asnos,...|Pure-bred breedin...|  0101|Cavalos, asininos...|Caballos, asnos, ...|Live horses, asse...|    01|Animais vivos|Animales vivos|Live animals|      

In [24]:
import pyspark.sql.functions

In [25]:
NCM.createOrReplaceTempView("ncm")

In [48]:
ncm_sh_ptbr = spark.sql("select CO_SH6, NO_SH6_POR as NO_SH6, CO_SH4, NO_SH4_POR as NO_SH4, CO_SH2, NO_SH2_POR as NO_SH2, CO_NCM_SECROM, NO_SEC_POR as NO_SEC from ncm ")
ncm_sh_ptbr.show(20, False)
ncm_sh_ptbr.write.parquet("s3a://silver/NCM_SH")

+------+---------------------------------------------------------------------------------+------+------------------------------------------------------------------------------------------------------------+------+-------------+-------------+----------------------------------------+
|CO_SH6|NO_SH6                                                                           |CO_SH4|NO_SH4                                                                                                      |CO_SH2|NO_SH2       |CO_NCM_SECROM|NO_SEC                                  |
+------+---------------------------------------------------------------------------------+------+------------------------------------------------------------------------------------------------------------+------+-------------+-------------+----------------------------------------+
|10110 |Animais vivos das espécies cavalar, asinina e muar, reprodutores de raça pura    |0101  |Cavalos, asininos e muares, vivos                     

In [47]:
ncm_sh_esp = NCM[['CO_SH6', 'NO_SH6_ESP', 'CO_SH4', 'NO_SH4_ESP', 'CO_SH2', 'NO_SH2_ESP', 'CO_NCM_SECROM', 'NO_SEC_ESP']]
ncm_sh_esp.show()
ncm_sh_esp.write.parquet("s3a://silver/NCM_SH_ESP")

+------+--------------------+------+--------------------+------+--------------+-------------+--------------------+
|CO_SH6|          NO_SH6_ESP|CO_SH4|          NO_SH4_ESP|CO_SH2|    NO_SH2_ESP|CO_NCM_SECROM|          NO_SEC_ESP|
+------+--------------------+------+--------------------+------+--------------+-------------+--------------------+
| 10110|Caballos y asnos,...|  0101|Caballos, asnos, ...|    01|Animales vivos|            I|Animales vivos y ...|
| 10111|Caballos reproduc...|  0101|Caballos, asnos, ...|    01|Animales vivos|            I|Animales vivos y ...|
| 10119|Demás caballos, v...|  0101|Caballos, asnos, ...|    01|Animales vivos|            I|Animales vivos y ...|
| 10120|Asnos, mulos y bu...|  0101|Caballos, asnos, ...|    01|Animales vivos|            I|Animales vivos y ...|
| 10121|Caballos de raza ...|  0101|Caballos, asnos, ...|    01|Animales vivos|            I|Animales vivos y ...|
| 10129|Caballos vivos, e...|  0101|Caballos, asnos, ...|    01|Animales vivos| 

In [46]:
ncm_sh_ing = NCM[['CO_SH6', 'NO_SH6_ING', 'CO_SH4', 'NO_SH4_ING', 'CO_SH2', 'NO_SH2_ING', 'CO_NCM_SECROM', 'NO_SEC_ING']]
ncm_sh_ing.show()
ncm_sh_ing.write.parquet("s3a://silver/NCM_SH_ING")

+------+--------------------+------+--------------------+------+------------+-------------+--------------------+
|CO_SH6|          NO_SH6_ING|CO_SH4|          NO_SH4_ING|CO_SH2|  NO_SH2_ING|CO_NCM_SECROM|          NO_SEC_ING|
+------+--------------------+------+--------------------+------+------------+-------------+--------------------+
| 10110|Pure-bred breedin...|  0101|Live horses, asse...|    01|Live animals|            I|Lives animals; An...|
| 10111|Pure-bred breedin...|  0101|Live horses, asse...|    01|Live animals|            I|Lives animals; An...|
| 10119|   Other live horses|  0101|Live horses, asse...|    01|Live animals|            I|Lives animals; An...|
| 10120|Asses, mules and ...|  0101|Live horses, asse...|    01|Live animals|            I|Lives animals; An...|
| 10121|Pure-bred breedin...|  0101|Live horses, asse...|    01|Live animals|            I|Lives animals; An...|
| 10129|Live horses, exce...|  0101|Live horses, asse...|    01|Live animals|            I|Lives